In [0]:
pip install geopandas

In [0]:
%python

import pandas as pd
from datetime import datetime
import geopandas as gpd
from shapely import wkt, union_all, intersection


# Outdoor Vision : Outings

In [0]:
sgga_grid = gpd.read_file('raw/grille_passage_2021.geojson')

In [0]:
sgga_grid.head(1)

## Old

In [0]:
ov_file = "raw/CC_Gorges_Ardeche_2020_2024_0.gpkg"
outdoor_vision = gpd.read_file(ov_file)

In [0]:
#outdoor_vision.provider.drop_duplicates()
#outdoor_vision.activity.drop_duplicates()
red_outdoor = outdoor_vision[['activity','user_id','date_start','date_end','geometry']]
red_outdoor.head(1)

In [0]:
red_outdoor.date_start.min()

In [0]:
red_outdoor.shape

# Biodiv : Sensitive Areas

In [0]:
biodiv = gpd.read_file("https://object.files.data.gouv.fr/dataeng-open/hackathon_outdoor/SIG/03-Biodiversite/Data_biodiv/data_geonatureSGGA.geojson")

In [0]:
biodiv['species_age'] = 'To Fill'
biodiv.loc[biodiv.stade_vie.isin(['Imago','Adulte']),'species_age'] = 'Adulte'
biodiv.loc[biodiv.stade_vie.isin(['Inconnu','Indéterminé',None]),'species_age'] = 'Inconnu'
biodiv.loc[biodiv.stade_vie.isin(['Exuvie','Larve','Juvénile','Œuf']),'species_age'] = 'Juvénile'
biodiv.loc[biodiv.stade_vie.isin(['Immature']),'species_age'] = 'Immature'

biodiv['behaviour'] = 'To Fill'
biodiv.loc[biodiv.comportement.isin(['Inconnu','Non renseigné',None]),'behaviour'] = 'Inconnu'
biodiv.loc[biodiv.comportement.isin(['Chasse/alimentation','Déplacement','Passage en vol','Hivernage','Repos','Halte migratoire']),'behaviour'] = 'Quotidien'
biodiv.loc[biodiv.comportement.isin(['Accouplement','Territorial','Tandem','Pond','Chant','Dortoir']),'behaviour'] = 'Territorialité'

In [0]:
to_keep = ['cd_nom', 'group2_inpn', 'niveau_sensibilite',
           'nom_valide', 
           'date_debut', 'date_fin', 
           'behaviour', 'species_age', 
           #'nombre_max', 'nombre_min', 'champs_additionnels',
           'geometry'
           # ,'x_centroid_4326', 'y_centroid_4326', 'geometrie_wkt_4326'
           ]
cleaned_biodiv = biodiv[(biodiv.regne == 'Animalia')
                        &(biodiv.date_debut >= '2020-01-01')
                        &(biodiv.date_debut <= '2024-12-31')][to_keep]
cleaned_biodiv['year_date'] = cleaned_biodiv['date_debut'].apply(lambda x : x.year)
cleaned_biodiv['year_month'] = cleaned_biodiv['date_debut'].apply(lambda x : x.month)
gpd.GeoDataFrame.to_crs(cleaned_biodiv,crs="EPSG:2154")
#cleaned_biodiv = gpd.GeoDataFrame(cleaned_biodiv, crs="EPSG:3857") #3857
cleaned_biodiv.head(3)

In [0]:
cleaned_biodiv.to_file('processed/cleaned_biodiv.geojson', driver="GeoJSON")

In [0]:
buffer_default = 0.002
cleaned_biodiv['geometry'] = cleaned_biodiv['geometry'].buffer(buffer_default)

In [0]:
to_group = ['cd_nom', 'nom_valide', 'group2_inpn', 'niveau_sensibilite', 
            'year_date', 'year_month',
            #'date_debut', 'date_fin', 
            'behaviour', 'species_age']
biodiv_agg = cleaned_biodiv.groupby(to_group).agg({'geometry':lambda x: list(x)}).reset_index()
biodiv_agg.head(3)

In [0]:
biodiv_agg = biodiv_agg.groupby(to_group).apply(lambda x: union_all(list(x.geometry)))\
                       .reset_index(name='geometry')
biodiv_agg = gpd.GeoDataFrame(biodiv_agg, crs="EPSG:2154") #3857

In [0]:
biodiv_agg.head(1)

In [0]:
biodiv_agg.plot()

In [0]:
biodiv_agg.to_file('processed/biodiv_agg.geojson', driver="GeoJSON")
#biodiv_agg.to_csv('processed/biodiv_agg.csv', index=False)  

In [0]:
biodiv_agg.head(1)

# Intersects

In [0]:
sgga_grid.shape

In [0]:
sgga_red = sgga_grid.head(10000) # sample for perf issues

In [0]:
sgga_red.plot()

In [0]:
biodiv_agg.plot()

In [0]:
in_biodiv = biodiv_agg.join(sgga_red, lsuffix='_bs', rsuffix='_otd', how='cross')
in_biodiv['intersects'] = in_biodiv.apply(lambda x: x.geometry_bs.intersects(x.geometry_otd), axis=1)
in_biodiv_red = in_biodiv[in_biodiv['intersects']==True].reset_index(drop=True)

In [0]:
in_biodiv_red['join_id'] = in_biodiv_red.index

In [0]:
biodiv_part = in_biodiv_red[['join_id','cd_nom', 'nom_valide', 'group2_inpn', 'niveau_sensibilite',
                             'year_date', 'year_month', 'behaviour', 'species_age', 'geometry_bs']]\
                           .rename(columns={"geometry_bs": "geometry"})
routes_part = in_biodiv_red[['join_id','n_passages', 'geometry_otd']]\
                           .rename(columns={"geometry_otd": "geometry"})

In [0]:
routes_part.shape

In [0]:
#biodiv_part[biodiv_part.join_id <= 1000].to_file('processed/biodiv_intersects_1.geojson', driver="GeoJSON")
#routes_part.to_file('processed/routes_intersects.geojson', driver="GeoJSON")
biodiv_part.to_parquet('processed/biodiv_intersects.parquet')

In [0]:
routes_part.plot() # our logo !